In [1]:
from requests import get

In [2]:
from bs4 import BeautifulSoup as BS

In [3]:
import re

In [68]:
import pandas as pd

In [4]:
adresse = "https://fr.wikipedia.org/wiki/CAC_40"

In [5]:
page_brute = get(url=adresse)

In [6]:
texte = page_brute.text

In [8]:
soupe = BS(texte, "lxml")

# Exercice

- Regarder les méthodes de l'objet `soupe`. (qui représente un arbre)
- Utiliser ces méthodes pour récupérer les données des entreprises du CAC40
- Construire un `dataframe` pandas regroupant les données.

## Exemple "bateau"

```
<balise1>  
premier texte  
<balise2>  
texte2  
</balise2>  
<balise3>  
texte3  
</balise3>  
</balise1>  
```
correspond à la structure sommet `balise1` relié au deux sommets `balise2` et `balise3`, ces sommets/noeuds ont des attributs par exemple le texte contenu.

In [35]:
for nom in dir(soupe):
    if "find_" in nom:
        print(nom)

_find_all
_find_one
find_all
find_all_next
find_all_previous
find_next
find_next_sibling
find_next_siblings
find_parent
find_parents
find_previous
find_previous_sibling
find_previous_siblings


**On pourra commencer par regarder le fonctionnement des méthodes basées sur** `find`

In [10]:
help(soupe.find_next)

Help on method find_next in module bs4.element:

find_next(name=None, attrs={}, text=None, **kwargs) method of bs4.BeautifulSoup instance
    Returns the first item that matches the given criteria and
    appears after this Tag in the document.



In [11]:
help(soupe)

Help on BeautifulSoup in module bs4 object:

class BeautifulSoup(bs4.element.Tag)
 |  This class defines the basic interface called by the tree builders.
 |  
 |  These methods will be called by the parser:
 |    reset()
 |    feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    handle_starttag(name, attrs) # See note about return value
 |    handle_endtag(name)
 |    handle_data(data) # Appends to the current data node
 |    endData(containerClass=NavigableString) # Ends the current data node
 |  
 |  No matter how complicated the underlying parser is, you should be
 |  able to build a tree using 'start tag' events, 'end tag' events,
 |  'data' events, and "done with data" events.
 |  
 |  If you encounter an empty-element tag (aka a self-closing tag,
 |  like HTML's <br> tag), call handle_starttag and then
 |  handle_endtag.
 |  
 |  Method resolution order:
 |      BeautifulSoup
 |      bs4.element.Tag
 |      bs4.element.PageElement
 |

In [13]:
balises_a = soupe.find_all(name="a")
type(balises_a[0])

bs4.element.Tag

In [16]:
balises_tables = soupe.find_all(name="table")
print(len(balises_tables))

8


In [19]:
balises_tables = soupe.find_all(name="table", attrs={"class" : "wikitable"})
print(len(balises_tables))

3


In [29]:
balises_tables = soupe.find_all(name="table", attrs={"class" : ["wikitable", "sortable"]})
print(len(balises_tables))

3


In [25]:
table1, table2, table3 = balises_tables

In [31]:
print(str(table1)[:500])

<table class="wikitable sortable">
<tbody><tr>
<th width="15%">Société<sup class="reference" id="cite_ref-valeur_54-0"><a href="#cite_note-valeur-54"><span class="cite_crochet">[</span>22<span class="cite_crochet">]</span></a></sup>
</th>
<th width="20%">Secteur
</th>
<th data-sort-type="number" width="9%">Poids indiciel au 30/06/2020 (en %)<sup class="reference" id="cite_ref-55"><a href="#cite_note-55"><span class="cite_crochet">[</span>23<span class="cite_crochet">]</span></a></sup>
</th>
<th 


In [32]:
print(str(table2)[:500])

<table border="3" class="wikitable sortable">
<tbody><tr>
<th>Date
</th>
<th>Valeur entrante
</th>
<th>Valeur sortante
</th></tr>
<tr>
<td>21 septembre 2020<sup class="reference" id="cite_ref-78"><a href="#cite_note-78"><span class="cite_crochet">[</span>26<span class="cite_crochet">]</span></a></sup>
</td>
<td><a href="/wiki/Alstom" title="Alstom">Alstom</a>
</td>
<td><a href="/wiki/Accor" title="Accor">Accor</a>
</td></tr>
<tr>
<td>22 juin 2020<sup class="reference" id="cite_ref-79"><a href="#


In [33]:
print(str(table3)[:500])

<table cellpadding="2" cellspacing="2" class="wikitable sortable gauche" style="text-align: center; font-size: 90%;">
<tbody><tr>
<th>Année
</th>
<th>Valeur
</th>
<th>Performance
</th>
<th>
</th>
<th>Valeur
</th>
<th>Performance
</th>
<th>
</th>
<th>Valeur
</th>
<th>Performance
</th></tr>
<tr>
<td>1987
</td>
<td>1 000,00
</td>
<td>-
</td>
<td>
</td>
<td>1 000,00
</td>
<td>-
</td>
<td>
</td>
<td>1 000,00
</td>
<td>-
</td></tr>
<tr>
<td>1988
</td>
<td>1 573,94
</td>
<td>+ 57,4 %
</td>
<td>
</td>
<


**On peut constater que la table1 est le meilleur candidat.**

- Récupérer le noeud correspondant au corps de la table.
- Dans ce sous arbre/noeud récupérer la liste des lignes via les balises `tr`
- Coder une fonction qui prend en entrée un noeud `tr` et qui renvoit la liste des attributs qui nous intéresse.
- Générer le `dataframe`

In [36]:
corps = table1.find_next(name="tbody")

In [37]:
print(str(corps)[:500])

<tbody><tr>
<th width="15%">Société<sup class="reference" id="cite_ref-valeur_54-0"><a href="#cite_note-valeur-54"><span class="cite_crochet">[</span>22<span class="cite_crochet">]</span></a></sup>
</th>
<th width="20%">Secteur
</th>
<th data-sort-type="number" width="9%">Poids indiciel au 30/06/2020 (en %)<sup class="reference" id="cite_ref-55"><a href="#cite_note-55"><span class="cite_crochet">[</span>23<span class="cite_crochet">]</span></a></sup>
</th>
<th data-sort-type="number" width="9%">


## ATTENTION

Il y a une différence entre ce qu'indique l'inspecteur (qui voit le code html après exécution du javascript) et ce qui est contenu dans le code brut html de la page.

In [70]:
titres, *lignes = corps.find_all(name="tr")

In [40]:
print(len(lignes))

40


In [41]:
ligne1, *_ = lignes

In [43]:
colonnes = ligne1.find_all(name="td")
for colonne in colonnes:
    print(colonne)
    print()

<td><a href="/wiki/Accor" title="Accor">Accor</a>
</td>

<td><a href="/wiki/H%C3%B4tellerie" title="Hôtellerie">Hôtellerie</a>, <a href="/wiki/Tourisme" title="Tourisme">Tourisme</a>
</td>

<td>0,41
</td>

<td>4 049
</td>

<td>7,05
</td>

<td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
</td>



In [44]:
nom, secteurs, poids_indiciel, chiffre_affaire, capitalisation, date_entree = colonnes

In [45]:
nom.find_next(name="a").text

'Accor'

In [53]:
[balise.text for balise in secteurs.find_all(name="a")]

['Hôtellerie', 'Tourisme']

In [60]:
float(poids_indiciel.text.strip().replace(",", "."))

0.41

In [61]:
int(chiffre_affaire.text.strip().replace(" ", ""))

4049

In [62]:
float(capitalisation.text.strip().replace(",", "."))

7.05

In [52]:
date_entree.find_next(name="time").attrs["data-sort-value"]

'1987-12-31'

## Assemblage en une fonction

In [65]:
def gestion_ligne(balise_tr):
    nom, secteurs, poids_indiciel, chiffre_affaire, capitalisation, date_entree =balise_tr.find_all(name="td")
    return [
        nom.find_next(name="a").text,
        [balise.text for balise in secteurs.find_all(name="a")],
        float(poids_indiciel.text.strip().replace(",", ".")),
        int(chiffre_affaire.text.strip().replace(" ", "")),
        float(capitalisation.text.strip().replace(",", ".")),
        date_entree.find_next(name="time").attrs["data-sort-value"]
    ]

In [67]:
table_liste = [gestion_ligne(ma_ligne) for ma_ligne in lignes]
for ligne_courante in table_liste:
    print(ligne_courante)

['Accor', ['Hôtellerie', 'Tourisme'], 0.41, 4049, 7.05, '1987-12-31']
['Air liquide', ['Gaz industriel'], 5.41, 21920, 66.84, '1987-12-31']
['Airbus', ['Aéronautique', 'Défense'], 3.31, 70478, 57.84, '1990-02-05']
['ArcelorMittal', ['Acier', 'Matériau de construction'], 0.56, 61737, 11.79, '2003-11-21']
['Atos', ['Technologies', 'Services informatiques'], 0.67, 12258, 7.86, '2017-03-20']
['Axa', ['Assurance', 'Services financiers'], 3.21, 103532, 41.85, '1987-12-31']
['BNP Paribas', ['Banque', 'Services financiers', 'Assurance'], 3.56, 44597, 46.74, '1987-12-31']
['Bouygues', ['BTP', 'Immobilier', 'Médias', 'Télécommunications'], 0.61, 37929, 13.06, '1987-12-31']
['Capgemini', ['Services informatiques'], 1.45, 14125, 20.05, '1988-08-03']
['Carrefour', ['Distribution'], 0.74, 80672, 11.68, '1987-12-31']
['Crédit agricole', ['Banque', 'Services financiers', 'Assurance'], 0.87, 20339, 25.22, '1999-09-07']
['Danone', ['Agroalimentaire'], 3.59, 25290, 38.99, '1987-12-31']
['Dassault Système

In [69]:
pd.DataFrame(table_liste)

,0,1,2,3,4,5
0,Accor,"[Hôtellerie, Tourisme]",0.41,4049,7.05,1987-12-31
1,Air liquide,[Gaz industriel],5.41,21920,66.84,1987-12-31
2,Airbus,"[Aéronautique, Défense]",3.31,70478,57.84,1990-02-05
3,ArcelorMittal,"[Acier, Matériau de construction]",0.56,61737,11.79,2003-11-21
4,Atos,"[Technologies, Services informatiques]",0.67,12258,7.86,2017-03-20
5,Axa,"[Assurance, Services financiers]",3.21,103532,41.85,1987-12-31
6,BNP Paribas,"[Banque, Services financiers, Assurance]",3.56,44597,46.74,1987-12-31
7,Bouygues,"[BTP, Immobilier, Médias, Télécommunications]",0.61,37929,13.06,1987-12-31
8,Capgemini,[Services informatiques],1.45,14125,20.05,1988-08-03
9,Carrefour,[Distribution],0.74,80672,11.68,1987-12-31


# Exercice

- **Nettoyer** et **réorganiser** le code pour obtenir un fichier .py avec une fonction `main` qui renvoit le dataframe final.
- Rajouter le nom et les unités de colonne (en les récupérant dans l'objet `table1`).
- Transformer la dernière colonne en "vraie" date.
- Découper la colonne secteurs en plusieurs colonnes.
- Rajouter une colonne contenant le nombre d'employés.
- Rajouter une colonne contenant le nom du pdg actuel.
